In [77]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pyzbar.pyzbar as pyzbar
import copy
# --------------- pre-processing --------------------
def pre_processing(image):
    h, w = image.shape[:2]
    #image = cv2.resize(image, (int(w/2), int(h/2)))
    """for i in range(0, h):
        for j in range(0, w):
            if image[i][j][0] <= 30 and image[i][j][1] <= 30 and image[i][j][2] <= 30:
                image[i][j] = 0,0,0
            elif image[i][j][0] >= 230 and image[i][j][1] >= 230 and image[i][j][2] >= 230:
                image[i][j] = 255,255,255"""
    return image

# -------------- find viga --------------------------
def find_viga(data):
    viga = [[], []]
    for d in data:
        if 'viga inicio' in str(d[0]):
            viga[0] = d[3]
        elif 'viga fim' in str(d[0]):
            viga[1] = d[3]
    return viga
            

# -------------- decode image -----------------------
def decode(image):
    qr_data = pyzbar.decode(image)
    return qr_data

def distance(a, b):
    return ((a[0]-b[0])**2+((b[1]-a[1])**2))**1/2

def process_image(viga, image):
    if viga[0][1] != viga[1][1]: #verifico de a altura dos qr codes é a mesma
        angle = np.sin(distance(viga[0][0], viga[0][1]))
        if abs(angle) > 0:
            cv2.rotate(image, int(angle))
    return image
    
def decode_all(image):
    results = decode(image)
    results_a = []
    for  r in results:
        if not 'viga inicio' in str(r[0]) and not 'viga fim' in str(r[0]):
            results_a.append(r)
    return results_a
    
        
    

# --------------- classify --------------------------
def classify(data):
    pontual = []
    distribuida = []
    triangular = []
    apoio1 = []
    apoio2 = []
    for d in data:
        if 'pontual' in str(d[0]):
            pontual.append(d)
        elif 'distr' in str(d[0]):
            distribuida.append(d)
        elif 'triang' in str(d[0]):
            triangular.append(d)
        elif 'apoio 1' in str(d[0]):
            apoio1.append(d)
        elif 'apoio 2' in str(d[0]):
            apoio2.append(d)
    return apoio1, apoio2, pontual, distribuida, triangular
        

# --------------- main ------------------------------
image = cv2.imread('img/triangular_10_new.png')[:,:,::-1]
image = pre_processing(image)
data = decode(image)
viga = find_viga(data)
image = process_image(viga, image)
all_data = decode_all(image)
classify(all_data)



classify(data)
#plt.imshow(image)


([],
 [Decoded(data=b't: apoio 2\nw: 0', type='QRCODE', rect=Rect(left=134, top=466, width=81, height=80), polygon=[Point(x=134, y=466), Point(x=134, y=546), Point(x=215, y=545), Point(x=215, y=466)]),
  Decoded(data=b't: apoio 2\nw: 0', type='QRCODE', rect=Rect(left=984, top=483, width=86, height=85), polygon=[Point(x=984, y=483), Point(x=984, y=568), Point(x=1070, y=568), Point(x=1069, y=483)])],
 [],
 [],
 [Decoded(data=b't: triang.\nw: 10', type='QRCODE', rect=Rect(left=707, top=24, width=72, height=72), polygon=[Point(x=707, y=24), Point(x=707, y=96), Point(x=779, y=96), Point(x=779, y=24)]),
  Decoded(data=b't: triang.\nw: 10', type='QRCODE', rect=Rect(left=421, top=364, width=76, height=74), polygon=[Point(x=421, y=364), Point(x=421, y=438), Point(x=497, y=438), Point(x=496, y=364)]),
  Decoded(data=b't: triang.\nw: 10', type='QRCODE', rect=Rect(left=719, top=372, width=79, height=79), polygon=[Point(x=719, y=372), Point(x=719, y=450), Point(x=798, y=451), Point(x=798, y=372)])]